In [40]:
#Importamos la librerías y variables necesarias
import mysql.connector
import sqlite3
import pandas as pd
import numpy as np
from flask import Flask, jsonify, request
import pickle
import os
#################################################################################
#################################################################################
"""Definimos las variables generales para atacar la base de datos"""
#Definimos un cursor con el que atacar a la DB
cnx = mysql.connector.connect(
    user="admin",
    password="admin123",
    host="test-db.cze2nnbbx5pc.eu-west-3.rds.amazonaws.com",
    database="prueba")
cursor = cnx.cursor()
#Definimos una función mediante la que ejecutar las querys
def make_query(code):
    cursor.execute(code)
    results = cursor.fetchall()
    cnx.commit()
    return(results)
#Definimos una función mediante la que ejecutar las querys devolviendo un dataframe
def make_query_dataframe(code):
    cursor.execute(code)
    results = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]  # Obtener los nombres de las columnas
    df = pd.DataFrame(results, columns=column_names)  # Crear el DataFrame
    return df

In [29]:
current_new=pd.read_table("trabajadores_actuales.csv")

In [30]:
current_new

,id_employee,name,involvement,performance,environment,satisfaction,life_balance,age,travel,department,...,marital_status,income,previous_companies,salary_hike,stock_option,total_working_years,training_ly,years_company,last_promotion,years_curr_manager
0,1,Jennifer Grant,High,Excellent,High,3,2,51,2,Sales,...,2,2148,1,11,0,1,6,1,0,0
1,3,Joel Rodgers,High,Excellent,Medium,5,4,32,0,Research & Development,...,1,1562,1,15,3,5,2,5,0,3
2,4,Gabriel Henson PhD,Medium,Excellent,Very High,4,1,38,0,Research & Development,...,1,2358,3,11,3,13,5,8,7,5
3,5,James Bryant,High,Excellent,Very High,7,1,32,0,Research & Development,...,1,1868,4,12,2,9,2,6,0,4
4,6,Michelle Wade,High,Excellent,High,9,1,46,0,Research & Development,...,1,2542,3,13,0,28,5,7,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3693,4405,Rebecca Carlson,Medium,Excellent,High,7,1,29,0,Sales,...,1,1456,1,18,0,6,2,6,1,5
3694,4406,Kimberly Tucker,High,Excellent,Very High,5,2,42,0,Research & Development,...,2,2147,3,17,1,10,5,3,0,2
3695,4407,Warren Phillips,Medium,Excellent,Very High,7,2,29,0,Research & Development,...,1,2625,2,15,0,10,2,3,0,2
3696,4408,Brendan Brown,High,Outstanding,Low,5,2,25,0,Research & Development,...,2,2585,0,20,0,5,4,4,1,2


In [38]:
#Creamos la query para la creación de la tabla y definimos los tipos de campos
make_query("""CREATE TABLE current_employees_2 (
    id_employee INT,
    name VARCHAR(50),
    involvement VARCHAR(50),
    performance VARCHAR(50),
    environment VARCHAR(50),
    satisfaction INT,
    life_balance INT,
    age INT,
    travel INT,
    department VARCHAR(50),
    distance_home INT,
    education VARCHAR(50),
    education_field VARCHAR(50),
    gender VARCHAR(50),
    job_level INT,
    role VARCHAR(50),
    marital_status INT,
    income INT,
    previous_companies INT,
    salary_hike INT,
    stock_option INT,
    total_working_years INT,
    training_ly INT,
    years_company INT,
    last_promotion INT,
    years_curr_manager INT,
    PRIMARY KEY (id_employee))"""
)

# Cargar los datos del CSV a la tabla
for row in current_new.itertuples(index=False):
    values = ', '.join("'" + str(value).replace("'", "''") + "'" for value in row)
    query = f"INSERT INTO current_employees_2 VALUES ({values})"
    cursor.execute(query)
    print(row)
# Confirmar los cambios en la base de datos
cnx.commit()
# Cerrar el cursor y la conexión
cursor.close()
cnx.close()

Pandas(id_employee=1, name='Jennifer Grant', involvement='High', performance='Excellent', environment='High', satisfaction=3, life_balance=2, age=51, travel=2, department='Sales', distance_home=6, education='College', education_field='Life Sciences', gender='Female', job_level=1, role='Healthcare Representative', marital_status=2, income=2148, previous_companies=1, salary_hike=11, stock_option=0, total_working_years=1, training_ly=6, years_company=1, last_promotion=0, years_curr_manager=0)
Pandas(id_employee=3, name='Joel Rodgers', involvement='High', performance='Excellent', environment='Medium', satisfaction=5, life_balance=4, age=32, travel=0, department='Research & Development', distance_home=17, education='Master', education_field='Other', gender='Male', job_level=4, role='Sales Executive', marital_status=1, income=1562, previous_companies=1, salary_hike=15, stock_option=3, total_working_years=5, training_ly=2, years_company=5, last_promotion=0, years_curr_manager=3)
Pandas(id_emp

In [39]:
old_employees=pd.read_table("trabajadores_historico.csv")

In [41]:
#Creamos la query para la creación de la tabla y definimos los tipos de campos
make_query("""CREATE TABLE old_employees_2 (
    id_employee INT,
    name VARCHAR(50),
    involvement VARCHAR(50),
    performance VARCHAR(50),
    environment VARCHAR(50),
    satisfaction INT,
    life_balance INT,
    age INT,
    travel INT,
    department VARCHAR(50),
    distance_home INT,
    education VARCHAR(50),
    education_field VARCHAR(50),
    gender VARCHAR(50),
    job_level INT,
    role VARCHAR(50),
    marital_status INT,
    income INT,
    previous_companies INT,
    salary_hike INT,
    stock_option INT,
    total_working_years INT,
    training_ly INT,
    years_company INT,
    last_promotion INT,
    years_curr_manager INT,
    PRIMARY KEY (id_employee))"""
)

# Cargar los datos del CSV a la tabla
for row in old_employees.itertuples(index=False):
    values = ', '.join("'" + str(value).replace("'", "''") + "'" for value in row)
    query = f"INSERT INTO old_employees_2 VALUES ({values})"
    cursor.execute(query)
    print(row)
# Confirmar los cambios en la base de datos
cnx.commit()
# Cerrar el cursor y la conexión
cursor.close()
cnx.close()

Pandas(id_employee=2, name='Brittany Dunn', involvement='Medium', performance='Outstanding', environment='High', satisfaction=4, life_balance=2, age=31, travel=0, department='Research & Development', distance_home=10, education='Below College', education_field='Life Sciences', gender='Female', job_level=1, role='Research Scientist', marital_status=2, income=1842.0, previous_companies=0, salary_hike=23, stock_option=0, total_working_years=6, training_ly=3, years_company=5, last_promotion=1, years_curr_manager=4)
Pandas(id_employee=7, name='Samuel Kim', involvement='High', performance='Outstanding', environment='Low', satisfaction=1, life_balance=1, age=28, travel=2, department='Research & Development', distance_home=13, education='College', education_field='College', gender='Medical', job_level=2, role='Sales Executive', marital_status=3, income=1554.0, previous_companies=2, salary_hike=20, stock_option=0, total_working_years=5, training_ly=2, years_company=0, last_promotion=0, years_cu